In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import gerarJsonCsv
import subprocess

In [2]:
planta = "sabesp-geral"
selected_gateways = None

In [3]:
# gerar o arquivo de parametros
gerarJsonCsv.generate_json_csv(planta)

/home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/gerarJsonCsv.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  modulos_data = pd.read_sql(f"\
/home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/gerarJsonCsv.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ativos_data = pd.read_sql(f"\


                  macaddress versao        aws_topic datetime_ultimo_envio  \
idModulo                                                                     
U2N000268  30:ed:a0:38:45:b4   3.00  U2N000268/dados                  None   
U2N000269  30:ed:a0:38:45:a8   3.00  U2N000269/dados                  None   
U2N000270  30:ed:a0:38:45:5c   3.00  U2N000270/dados                  None   
U2N000271  fc:01:2c:d2:24:30   3.00  U2N000271/dados                  None   
U2N000272  fc:01:2c:d2:24:14   3.00  U2N000272/dados                  None   
...                      ...    ...              ...                   ...   
U2N000378  30:ed:a0:ad:0c:ac   3.00  U2N000378/dados                  None   
U2N000379  30:ed:a0:ad:0c:c8   3.00  U2N000379/dados                  None   
U2N000380  30:ed:a0:ad:0c:a0   3.00  U2N000380/dados                  None   
U2N000381  30:ed:a0:ad:0c:fc   3.00  U2N000381/dados                  None   
U2N000382  30:ed:a0:ad:19:90   3.00  U2N000382/dados            

In [4]:
mydb = mysql.connector.connect(
    host="ultronline.cibxpi1gohmx.us-east-1.rds.amazonaws.com",
    user="root",
    password="2neuron2",
    database='projeto_ultron_teste'
)
mycursor = mydb.cursor()

In [5]:
gateways_data = pd.read_sql(f"\
SELECT * FROM Gateways where Plantas_idPlanta='{planta}'", mydb)

/tmp/ipykernel_143089/412794979.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gateways_data = pd.read_sql(f"\


In [6]:
gateways_data

,idGateway,wifi_ssid,wifi_passkey,mqtt_ip,mqtt_user,mqtt_passkey,mqtt_port,cript_key,rasp_user,rasp_passkey,chip_numero,aws_thing,ipv6,macaddress,key_credenciais,Plantas_idPlanta,Real_vnc_login
0,GW000102,GW000102,mQ7mBCaM,10.42.0.1,GW000102,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,9t3keOhC,None,GW000102,2804:18:846:acea:6800:93af:7d60:d868,None,None,sabesp-geral,não@tem
1,GW000103,GW000103,mQ7mBCaM,10.42.0.1,GW000103,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,U9HrzXGf,None,GW000103,2804:18:db:8c33:7e32:94a4:2417:482,None,None,sabesp-geral,não@tem
2,GW000104,GW000104,mQ7mBCaM,10.42.0.1,GW000104,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,0aPjPk9y,None,GW000104,2804:18:8c2:44ad:f810:6550:1034:d9d4,None,None,sabesp-geral,não@tem
3,GW000105,GW000105,mQ7mBCaM,10.42.0.1,GW000105,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,mR32PZvT,None,GW000105,2804:18:10c8:44f1:4744:5f0a:5e50:ac7d,None,None,sabesp-geral,não@tem
4,GW000106,GW000106,mQ7mBCaM,10.42.0.1,GW000106,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,qmrnVorD,None,GW000106,2804:18:865:63c4:13f4:11fb:7b51:167d,None,None,sabesp-geral,não@tem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,GW000161,GW000161,mQ7mBCaM,10.42.0.1,GW000161,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,bvymlmDJ,None,GW000161,2804:38a:a01a:5425:dc2b:357e:ae03:b4db,None,None,sabesp-geral,não@tem
59,GW000162,GW000162,mQ7mBCaM,10.42.0.1,GW000162,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,GGfAoSzt,None,GW000162,2804:18:93b:dcf:4615:c537:117c:a332,None,None,sabesp-geral,não@tem
60,GW000163,GW000163,mQ7mBCaM,10.42.0.1,GW000163,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,2XGjUvML,None,GW000163,2804:18:18c6:f171:5387:a3ea:c288:21a1,None,None,sabesp-geral,não@tem
61,GW000164,GW000164,mQ7mBCaM,10.42.0.1,GW000164,0gyinBuH,1883,SrrJrvcXS4bFTtMj,gateway,WSUIKukb,None,GW000164,2804:18:845:f4ac:a9dd:7517:9004:d13a,None,None,sabesp-geral,não@tem


In [7]:
file_text = f"[myhosts]\n"
add_servers_text = ""
for i,row in gateways_data.iterrows():
    idGateway = row["idGateway"]
    ipv6 = row["ipv6"]
    rasp_passkey = row["rasp_passkey"]
    if selected_gateways is None:
        add_servers_text += f"\
servidor_{idGateway} ansible_host={ipv6} ansible_port=30022 ansible_user=gateway ansible_ssh_pass={rasp_passkey} \
ansible_become_pass={rasp_passkey} ansible_ssh_common_args='-6' StrictHostKeyChecking=no\n"
    else:
        if idGateway in selected_gateways:
            add_servers_text += f"\
servidor_{idGateway} ansible_host={ipv6} ansible_port=30022 ansible_user=gateway ansible_ssh_pass={rasp_passkey} \
ansible_become_pass={rasp_passkey} ansible_ssh_common_args='-6' StrictHostKeyChecking=no\n"

In [8]:
print(add_servers_text)

servidor_GW000102 ansible_host=2804:18:846:acea:6800:93af:7d60:d868 ansible_port=30022 ansible_user=gateway ansible_ssh_pass=9t3keOhC ansible_become_pass=9t3keOhC ansible_ssh_common_args='-6' StrictHostKeyChecking=no
servidor_GW000103 ansible_host=2804:18:db:8c33:7e32:94a4:2417:482 ansible_port=30022 ansible_user=gateway ansible_ssh_pass=U9HrzXGf ansible_become_pass=U9HrzXGf ansible_ssh_common_args='-6' StrictHostKeyChecking=no
servidor_GW000104 ansible_host=2804:18:8c2:44ad:f810:6550:1034:d9d4 ansible_port=30022 ansible_user=gateway ansible_ssh_pass=0aPjPk9y ansible_become_pass=0aPjPk9y ansible_ssh_common_args='-6' StrictHostKeyChecking=no
servidor_GW000105 ansible_host=2804:18:10c8:44f1:4744:5f0a:5e50:ac7d ansible_port=30022 ansible_user=gateway ansible_ssh_pass=mR32PZvT ansible_become_pass=mR32PZvT ansible_ssh_common_args='-6' StrictHostKeyChecking=no
servidor_GW000106 ansible_host=2804:18:865:63c4:13f4:11fb:7b51:167d ansible_port=30022 ansible_user=gateway ansible_ssh_pass=qmrnVorD

In [9]:
with open(f"inventory-{planta}.ini", "w") as f:
    f.write(file_text + add_servers_text)

In [10]:
# escrever o playbook
playbook_text = f"""\
---
- name: move files
  hosts: myhosts
  become: true  # se precisar de sudo
  tasks:
  - name: Move ultronline.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/ultronline.py
      dest: /home/gateway/Servidores/server_v4/Server/ultronline.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move servidor_main.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/servidor_main.py
      dest: /home/gateway/Servidores/server_v4/Server/servidor_main.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move energy_analysis.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/energy_analysis.py
      dest: /home/gateway/Servidores/server_v4/Server/energy_analysis.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move parametros_ultronline.json
    ansible.builtin.copy:
      src: /home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/\
parametros_ultronline_{planta}.json
      dest: /home/gateway/Servidores/server_v4/Server/parametros_ultronline.json
      owner: root
      group: root
      mode: '0644'
  
  - name: Reiniciar ultronlink_server
    ansible.builtin.service:
      name: ultronlink_server
      state: restarted
"""

In [11]:
with open(f"playbook-{planta}.yml", "w") as f:
    f.write(playbook_text)

export ANSIBLE_HOST_KEY_CHECKING=False

In [ ]:
print(f"ansible-playbook -i inventory-{planta}.ini playbook-{planta}.yml")

ansible-playbook -i inventory-sabesp-geral.ini playbook-sabesp-geral.yml


In [ ]:
mycursor.close()
mydb.close()

In [ ]:
# TCs ruins
# -> U2N000313
# 